## 管理层信息抽取

In [ ]:
! cp /Users/alchemy_taotaox/Desktop/next/my_test/* ../corpus/

In [ ]:
! ls ../corpus

## 算法基本思路
1. 扫描pdf 获得页眉页脚
2. 扫描pdf 获得最小和最大的x坐标
    如果一个字符是最小的坐标，那么就和之前的行合并为一个段落
    如果一个字符是最大的坐标，那么就和之前的
2. 提取文档中的表格和字符串。

In [ ]:
from pdfplumber.pdf import PDF
import re
from collections import Counter

def is_header(str, header):
    if header is None:
        return False
    return str == header

def is_foot(str, foot):
    if foot is None:
        return False
    
    s = re.sub(r'\d+', '#num#', str)
    return s == foot

def get_foot_header(pdf:PDF, check_pages_num=20, threshold = 0.9):
    head_counter = Counter()
    foot_counter = Counter()
    pages_num = len(pdf.pages)
    if pages_num < check_pages_num:
        check_pages_num = pages_num
    for i in range(check_pages_num):
        text_lines = pdf.pages[i].extract_words()
        top_line = text_lines[0]['text']
        bottom_line = text_lines[-1]['text']
        bottom_line = re.sub(r'\d+', '#num#', bottom_line)
        head_counter.update([top_line])
        foot_counter.update([bottom_line])
        
    most_common_head_word, count_head = head_counter.most_common(1)[0]
    most_common_foot_word, count_foot = foot_counter.most_common(1)[0]
    head, foot = None, None
    if count_head *1./check_pages_num >= threshold:
        head = most_common_head_word
    if count_foot *1./check_pages_num >= threshold:
        foot = most_common_foot_word
    return head, foot
    
    

In [ ]:
import pdfplumber

In [116]:
from operator import itemgetter
import json

def check_bboxes(word, table_bbox):
    """
    Check whether word is inside a table bbox.
    """
    l = word['x0'], word['top'], word['x1'], word['bottom']
    r = table_bbox
    return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]



def get_lines_from_page(page, head, foot):
    tables = page.find_tables()
    table_bboxes = [i.bbox for i in tables]
    tables = [{'table': i.extract(), 'top': i.bbox[1]} for i in tables]
    non_table_words = [word for word in page.extract_words() if not any(
        [check_bboxes(word, table_bbox) for table_bbox in table_bboxes])]
    lines = []
    lines_x0_x1 =[]
    for cluster in pdfplumber.utils.cluster_objects(
        non_table_words + tables, itemgetter('top'), tolerance=5):
        if 'text' in cluster[0]:
            x_0 = x_1 = -1
            inner_lines = []
            for item in cluster:
                if len(cluster) == 1 and (is_foot(item['text'], foot) or  is_header(item['text'], head)):
                    continue
                # print(item, cluster)
                if x_0 == -1:
                    if 'x0' in item:
                        x_0 = item['x0']
                if 'x1' in item:
                    x_1 = item['x1']
                if 'text' in item:
                    inner_lines.append(item['text'])
                elif 'table' in item:
                    inner_lines.append("\n" + json.dumps(item['table'], ensure_ascii=False) + "\n")
            if len(inner_lines) > 0:
                lines.append(' '.join(inner_lines))
                lines_x0_x1.append((x_0, x_1))
                
            # lines.append(' '.join([i['text'] for i in cluster if not is_foot(i['text'], foot) and not is_header(i['text'], head)]))
        elif 'table' in cluster[0]:
            lines.append(cluster[0]['table'])
            lines_x0_x1.append((10000000, -1))

    # Find the minimum of the first elements
    if len(lines_x0_x1) == 0:
        return []
    min_first = min(x[0] for x in lines_x0_x1)

    # Find the maximum of the second elements
    max_second = max(x[1] for x in lines_x0_x1)
       
    # print(lines)

    for index, item in enumerate(lines):
        if type(lines[index]) == str:
            lines[index] = lines[index].strip()
        # 如果锁进， 那么要换行
        
        if abs(lines_x0_x1[index][0] - min_first) > 0.1 and lines_x0_x1[index][0] < 1000000:
            # print(lines_x0_x1[index][0], min_first, "----")
            lines[index] = "\n" + lines[index]
        # 如果没有写完，那么换行
        if abs(lines_x0_x1[index][1] - max_second) > 16 and lines_x0_x1[index][1] > 0:
            # print(lines_x0_x1[index][1], max_second, "----", lines[index])
            lines[index] = lines[index] + "\n"
        elif lines_x0_x1[index][1] > 0:
            # print('******FULL LINE******', lines_x0_x1[index][1], max_second, "----", lines[index])
            pass
        # print(lines[index], end='')
    return lines

In [117]:
filename = "../corpus/year_report_2.pdf"
filename = "/tmp/a.PDF"
pdf = pdfplumber.open(filename)
# page = pdf.pages[10]
# page = pdf.pages[15]
page = pdf.pages[3]
# print(page.extract_text())
head, foot = get_foot_header(pdf)
print(head)
print(foot)
get_lines_from_page(page, head, foot)

print(pdf.pages[3].extract_words() )# 计算一个字符大概的宽度是多少，然后定义magic distance = 16
print(pdf.pages[3].extract_text())

None
None
[{'text': '获批的交易', 'x0': 303.89, 'x1': 356.57383999999996, 'top': 76.2489599999999, 'doctop': 2602.0089599999997, 'bottom': 86.80895999999996, 'upright': True, 'direction': 1}, {'text': '是否超过交易', 'x0': 372.19, 'x1': 435.31768, 'top': 76.2489599999999, 'doctop': 2602.0089599999997, 'bottom': 86.80895999999996, 'upright': True, 'direction': 1}, {'text': '关联交易内', 'x0': 168.74, 'x1': 221.42384, 'top': 85.26895999999988, 'doctop': 2611.0289599999996, 'bottom': 95.82895999999994, 'upright': True, 'direction': 1}, {'text': '关联方', 'x0': 108.74, 'x1': 140.42, 'top': 94.26895999999988, 'doctop': 2620.0289599999996, 'bottom': 104.82895999999994, 'upright': True, 'direction': 1}, {'text': '本期发生额', 'x0': 236.57, 'x1': 289.25384, 'top': 94.26895999999988, 'doctop': 2620.0289599999996, 'bottom': 104.82895999999994, 'upright': True, 'direction': 1}, {'text': '额度（如适', 'x0': 303.89, 'x1': 356.57383999999996, 'top': 94.26895999999988, 'doctop': 2620.0289599999996, 'bottom': 104.82895999999994, 

In [120]:
from pdfplumber.pdf import PDF
import json
def begin_mda(lines):
    if len(lines) < 1:
        return False
    if type(lines[0]) != str:
        return False
    return lines[0].find('管理层讨论与分析') != -1
def end_mda(lines):
    if len(lines) < 1:
        return False
    if type(lines[0]) != str:
        return False
    # print("check: ", lines[0], "$$$$$$")
    return lines[0].find('公司治理') != -1

def get_all_lines_about_mda(pdf:PDF, head:str, foot:str):
    begin = False
    end = False
    all_lines = []
    for page in pdf.pages:
        lines = get_lines_from_page(page, head, foot)
        if begin is False and begin_mda(lines):
            begin = True
        if begin and end_mda(lines):
            break
        if begin is True:
            all_lines.append(lines)

    content_list = []
    for lines in all_lines:
        for line in lines:
            if type(line) != str:
                line = json.dumps(line, ensure_ascii=False)
            # print(line, end='')
            content_list.append(line)
    return "".join(content_list)
            
content = get_all_lines_about_mda(pdf, head, foot)
if content is not None:
    print(content)
else:
    print("None")

## 抓取信息



In [ ]:
from datetime import datetime, timedelta

def split_interval(start, end, day_cnt):
    date_format = "%Y-%m-%d"
    start_date = datetime.strptime(start, date_format)
    end_date = datetime.strptime(end, date_format)
    
    result = []

    while start_date < end_date:
        # Calculate the end date of the current interval.
        interval_end_date = start_date + timedelta(days=day_cnt-1)

        # Ensure that the interval doesn't go beyond the overall end date.
        if interval_end_date > end_date:
            interval_end_date = end_date

        # Append the current interval to the result list.
        result.append(
            (start_date.strftime(date_format), 
             interval_end_date.strftime(date_format), 
             (interval_end_date - start_date).days + 1)
        )

        # Move the start date to the next day after the current interval.
        start_date = interval_end_date + timedelta(days=1)

    return result
results = split_interval('2023-01-01', '2023-07-31', 1)

print(results)

In [ ]:
import requests
import time
from tqdm import tqdm
import json




def request_data(page_size=1000, page_num=1, se_date="2023-04-26~2023-04-26"):
    url = "http://www.cninfo.com.cn/new/hisAnnouncement/query"
    if page_num > 1:
        url = "http://www.cninfo.com.cn/new/hisAnnouncement/query"

    params = {
        "pageNum": page_num,
        "pageSize": page_size,
        "column": "szse",
        "tabName": "fulltext",
        "plate": "",
        "stock": "",
        "searchkey": "",
        "secid": "",
        "category": "category_ndbg_szsh",
        "trade": "",
        "seDate": se_date,
        "sortName": "",
        "sortType": "",
        "isHLtitle": "true"
    }

    response = requests.post(url, data=params)

    if response.status_code == 200:
        return response.json()  # 或者返回response.text取决于你需要什么样的数据格式
    else:
        return None

file = open('report_meta_info.txt', 'w')
results = split_interval('2023-01-01', '2023-07-31', 1)
results = results[::-1]
#for date_range in results:
for i, date_range in enumerate(tqdm(results)):
    se_date = f"{date_range[0]}~{date_range[1]}"
    
    page_size = 30
    data = request_data(page_size=10, se_date=se_date)
    try_times = 0
    while data is None:
        time.sleep(1)
        data = request_data(page_size=10, se_date=se_date)
        try_times += 1
        print(f"craw data wrong, try {try_times}")
        if try_times == 3:
            break  
    if data is None:
        continue
    page_count = (data['totalAnnouncement'] -1 )// page_size  + 1
    print(page_count)
    begin = time.time()
    for index in range(1, page_count+1):
    
        data = request_data(page_num=index, page_size=page_size, se_date=se_date)
        try_times = 0
        while data is None:
        
            time.sleep(3)
            data = request_data(page_num=index, page_size=page_size, se_date=se_date)
            try_times += 1
            print(f"craw data wrong, try {try_times}")
            if try_times == 3:
                break
        
        last = time.time() - begin
        need_time = last/index * page_count  - last
        every_time = last*1./index
        print(f"{index}/{page_count}: last time {last} 秒， need {need_time}, every page cost: {every_time}, count: { len(data['announcements'])}, but page size is {page_size}")
        for item in data['announcements']:
            file.write(json.dumps(item, ensure_ascii=False) + "\n")
file.close()
    

In [121]:
from urllib.parse import urljoin
import os

def download_and_process_file(url, local_path='/tmp/a.PDF', try_count=3):
    try_times = 0
    response = None
    while True:
        # 下载文件
        response = requests.get(url)
        if response.status_code == 200:
            break
            # return response.json()  # 或者返回response.text取决于你需要什么样的数据格式
        else:
            response = None
        try_times += 1
        if try_times >= try_count:
            time.sleep(3)
            break
    if response is None:
        return response
    # response.raise_for_status()  # 确保请求成功

    # 将文件写入本地
    with open(local_path, 'wb') as output_file:
        output_file.write(response.content)

    # 读取PDF文件的内容
    
    pdf = pdfplumber.open(local_path)
    head, foot = get_foot_header(pdf)
    content = get_all_lines_about_mda(pdf, head, foot)
    #print(content)
    # 删除临时文件
    os.remove(local_path)

    return content
'''
1. read meta file
2. download pdf file into tmp file
3. get the necessary content
4. write data into file
   {'stock_code', 'stock_name', 'content':   'url':  'pubtime': xxx}

'''
base_url = 'http://static.cninfo.com.cn/'
meta_file = 'report_meta_info.txt'
data_file = 'report_data_info.txt'
file = open(data_file, "w")
lines = open(meta_file).readlines()
code_set = set()
for index, line in tqdm(enumerate(lines)):
    info = json.loads(line.strip())
    '''
    "secCode": "001211", "secName": "双枪科技, adjunctUrl
    '''
    title = info['announcementTitle']
    if title.find('关于') != -1 or title.find("摘要") != -1:
        continue
    stock_code = info['secCode']
    if stock_code in code_set:
        continue
    code_set.add(stock_code)
    stock_name = info['secName']
    relative_url = info['adjunctUrl']
    pub_time = relative_url.split('/')[-2]
    url =  urljoin(base_url, relative_url)
    content = download_and_process_file(url, "/tmp/a.PDF")
    result = {"title": title, 'pubtime': pub_time, 'stock_code': stock_code, 'stock_name': stock_name, 'content': content, 'url': url}
    file.write(json.dumps(result, ensure_ascii=False) + "\n")
    if index == 100:
        break

   
   

1it [00:07,  7.61s/it]


第三节 管理层讨论与分析
一、报告期内公司所处行业情况

公司定位日用餐厨具行业，主营产品为筷子、砧板等。日用餐厨具属于必备的日常用品，其市场与人口数量、家庭户数等高度相关。根据国家统计局数据显示，2022年末，我国人口总数约14.12亿，人口总量保持稳定。同时国家三孩生育政策的出台、积极应对人口老龄化等重大决策部署的实施，有利于推动人口数量的持续增长。2022年末我国常住人口城镇化率为65.22%，根据国家“十四五”规划纲要提出的目标，2025年我国常住人口城镇化率预计提高到65%，在加快城镇化进程的同时，提升城镇化发展质量。这意味着，我国城镇化率突破65%的目标已提前完成，接下去这个阶段人口会加速向中心城市聚集。受益中国庞大的人口基数以及城镇化建设加快，日用餐厨具行业规模不断扩大，发展长期向好。

日用餐厨具作为日常生活消费品，行业发展与经济水平高度相关。根据国家统计局数据显示，2022年社会消费品零售总额达43.97万亿元，比上年微降0.20%。受2022年大环境反复影响终端零售、物流运输以及生产仓储，公司所处日用餐厨具行业内销增速明显放缓。2022年12月，中共中央、国务院印发《扩大内需战略规划纲要（2022-2035年）》，明确要全面促进消费，加快消费提质升级。恢复和扩大消费在2023年将被摆在优先位置，有望带动日用餐厨具行业消费复苏。居民生活水平快速提升带动行业发展，根据国家统计局数据显示，2022年末我国人均可支配收入36,883.00元，比上年名义增长5.00%；居民年均消费支出24,538.00元，比上年名义增长1.80%；其中居民人均生活用品及服务消费支出1,432.00元，增长0.63%，占人均消费支出的比重为5.80%。人均收入提升带来居民消费水平的不断提高，促进消费结构升级，带动日用餐厨具行业向上发展。随着消费者的环保意识不断提升，对健康、绿色、安心的高品质餐厨具需求扩张。公司的竹木材质日用餐厨具取材天然，复合材料产品耐高温可消毒，植物纤维复合餐具环保可降解，抗菌系列产品作为健康新型环保的餐厨用品，迎来巨大的市场空间。

城镇化率提升、居民可支配收入增加，推动在外就餐人数以及频次增长。日用餐厨具作为就餐过程使用的必要工具，餐饮行业的快速发展将直接带动日用餐厨具的需求量增加，行业整体前景良好。

电子商务的快速发展大幅提升日用餐厨具企业产品

3it [00:18,  6.03s/it]


第三节 管理层讨论与分析
一、经营情况讨论与分析

国内经济活动正逐步恢复，公司主营业务面向人民群众基础性饮食大消费，市场需求强劲，面对国内庞大市场，公司采取积极措施提升主营业务产业链核心竞争力-从源头保证食品绿色安全、降低产品成本及价格、研发热销佳品、拓展市场覆盖和品牌建设，赢得消费者信赖，扩大市场规模和市场占用率。

报告期内，外部总体经营环境不佳-各地区内部及跨地区人员通行和商品物流受到限制，商业活动尤其是到店服务业受到极大影响，公司连锁餐饮和殡葬相关业务营业收入出现了一定程度的下滑；公司部分下游客户出现经营困难，为此报告期内公司增加计提了应收账款坏账准备；根据天德福地陵园非法吸收公众存款案（以下简称：“非吸案件”）处理和和辉基金清算的情况和进展，报告期内公司对其他非流动资产(对和辉基金的投资款)计提了资产减值损失，天德福地陵园对非吸案件可能产生的赔付义务计提了预计负债。公司主营业务面向人民饮食基础必需消费，国内市场基本需求依然稳定强劲，公司总体经营保持稳健。2022年公司实现营业收入10.74亿元和归属于上市公司股东的净利润1.07亿元，同比分别减少15.34%和28.62%；公司长期保持经营性现金流入，2022年公司经营性现金流入1.04亿元，同比增长100.70%；公司财务健康，报告期末公司有息负债为零，资产负债率由2021年19.41%下降至13.32%。

响应2023年中央1号文件精神-“大力推进种源等农业关键核心技术攻关”、“提升农机装备研发应用水平”和“加快发展设施农业”。2022 年生态农业业务未取得实质性进展，公司生态农业发展重点方向转向与肉牛育种养殖、食品制造加工和连锁餐饮业务结合配套，应用大型农机装备实施大规模自动化机械化种植，升级现代农业，建立饮食消费大产业链绿色源头；报告期内，公司全面完成燕郊西吴养牛场北区和南区的改扩建工程，建设工厂化集约养殖基地，容纳存栏活牛的规模超过1万头。公司将进一步扩大工厂化集约养殖基地规模，充分利用可开发空闲土地建设活牛育种养殖设施。利用公司多年肉牛养殖经验和优势，肉牛育种养殖屠宰业务实现高水平恢复，至报告期末已生产犊牛6,941头，肉牛育种养殖业务实现毛利润5,180.88万元，为公司进一步做强肉牛育种养殖业务迈出坚实步伐；公司将投入优质活牛种源培育工作，与科研院所及同行共同攻关解决农业领域的优质肉牛

4it [00:32,  8.60s/it]

...


第三节 管理层讨论与分析
一、报告期内公司所处行业情况

公司主要产品为手机等移动智能终端保护类配件，是消费电子产业链的组成部分。与移动智能终端产品共同受到移动通信、智能终端等技术的发展及相关政策的影响，同时以信息化、智能化为突出特点的移动智能终端产业受到国家产业政策的大力引导和支持，国家相关部门出台的一系列支持移动智能终端、移动通信产业发展。根据市场研究机构 IDC 发布的报告显示，2022 年全球智能手机出货量为 12.1 亿台，同比下降 11.3%，该出货量为 2013 年以来的最低年度出货量。此外，行业下游的市场环境持续发生变化，受全球主要贸易体经济摩擦，芯片市场供给及产品创新周期等多种因素影响，导致下游行业客户的市场格局发生显著变化，进而对移动智能终端的相关配件厂商带来一定的影响。目前，我国仍是全球智能手机最大的消费国和重要的生产国，未来空间依旧很大。随着5G时代的推进，智能手机、智能手表等各类智能终端产品的市场需求将持续扩大。相应地，与智能终端产品密切相关的配件产品的市场需求仍将稳定增长。公司是行业内较早从事移动智能终端配件的研发、设计、生产、销售业务的公司之一，核心管理人员均拥有多年的行业管理经验，对公司和行业的发展有深刻理解，在国内配件行业 ODM/OEM 领域中拥有较为明显的先发优势。经过多年的沉淀，公司建立起了以大型移动智能终端制造商和全球知名移动智能终端配件品牌商为主的 ODM/OEM 业务市场布局，在下游客户中建立了良好的产品口碑，拥有了一批稳定、优质的合作伙伴。通过移动智能终端制造商客户以及国际知名移动智能终端配件品牌商的合作，公司的经营资质、生产规模、技术水平、产品品质、货物交期、环境管理体系等多方面都经过了严格评估及验厂审核，成为众多行业内知名客户的重要供应商，也进一步印证了公司在细分领域的行业地位与品牌实力。随着公司募投项目的稳步推进，公司的核心竞争力将得到进一步提升。
二、报告期内公司从事的主要业务

公司是一家专注于移动智能终端保护类配件产品研发、设计、生产及销售的高新技术企业。自2006年成立以来，公司始终专注深耕细分行业，经过多年沉淀，公司已在产品研发设计、高精度模具开发、注塑生产自动化、质量体系标准化等方面积累了丰富的经验。公司也形成了以全球知名的移动智能终端设备厂商、细分行业领头品牌商、全球连锁商超等多维度的大客户结构。

7it [00:41,  5.09s/it]

10it [00:47,  3.36s/it]


第三节 管理层讨论与分析

一、报告期内公司所处行业情况

公司需遵守《深圳证券交易所上市公司自律监管指引第3号——行业信息披露》中纺织服装相关业的披露要求

（一）社会消费逐渐复苏，线下零售逐渐回暖

2022 年随着国家政策持续发力，居民消费逐渐复苏。2022 年国民经济生产总值（GDP）较 2021 年增长 5.31%；消
费者物价指数（CPI）及消费者衣着类物价指数（衣着类 CPI）于 2020 年出现下降现象，随着政策的发展及经济的复苏
2022年CPI和衣着类CPI逐渐趋于平稳，消费市场供需基本平稳，价格总体稳定。社会消费品零售总额、服装类商品零
售类值 2022年度发生波动，分别同比下降-1.56%和-7.88%。在政策支持下，公司线下零售店铺逐渐恢复正常运营。但国
内外形势依旧复杂严峻，全球消费能力与消费信心的削弱给公司盈利带来了巨大挑战。

图1：社会消费品零售总额
数据来源：同花顺iFind，摩登大道时尚集团

图2：服装鞋帽、针、纺织品类商品零售类值 图3：CPI及衣着类CPI
数据来源：同花顺iFind，摩登大道时尚集团 数据来源：同花顺iFind，摩登大道时尚集团
（二）服装行业显示发展韧性，品牌文化属性增强

服装作为居民生活必需品“衣食住行”中不可或缺的一部分，在经济下行期艰难维持行业活力，不断探索更多业务方
式，展现出了顽强、可持续的发展韧性。品牌的文化属性较以往更为显著，融合中国传统元素的设计更受消费者青睐，
企业履行的社会责任也纳入了产品选择的考量范围。

二、报告期内公司从事的主要业务

公司需遵守《深圳证券交易所上市公司自律监管指引第3号——行业信息披露》中纺织服装相关业的披露要求

（一）主要业务及产品介绍

公司主要业务为自有品牌卡奴迪路（CANUDILO）高级男装的研发设计、组织外包生产、品牌推广和终端销售，以
及开展国际一二线服装品牌的代理业务。公司秉持国际高端品牌运营理念，凭借公司长期积累的品牌运营经验，致力于
品牌资源整合、营销网络建设、品牌推广和供应链管理，不断提升自有品牌和国际代理品牌的标准化经营水平，打造成
为行业领先的全球时尚品牌运营商。

1、自有高级男装品牌经营业务

公司自有品牌卡奴迪路（CANUDILO）致力于以领先的时尚文化和设计理念为指引，将艺术性与实用性结合，推出
自然、简约、精致、时尚的服

10it [01:34,  9.40s/it]